In [1]:
import pickle as pkl
import pandas as pd
import os
import numpy as np
from collections import Counter
import json

with open('config.json', 'r') as f:
    config = json.load(f)
cwd = os.getcwd()
os.chdir(config['REPODIR'])
import Utils as U
from Corpus import Corpus
os.chdir(cwd)

/Users/stephentoner/miniconda3/envs/si699proj/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load both dataset and the set difference of previous selected works, then 

In [6]:
# os.chdir('..')

In [2]:
df_ff = U.load_file('data_vFF.pkl', 'pkl', config['DATADIR'])

In [3]:
df_ff = pd.DataFrame(df_ff)

Split data into sentence embeddings and author id

In [4]:
baby_size = int(df_ff.author_id.nunique() * .1 // 1)
baby_set = set(df_ff.author_id.sample(baby_size))
baby_df = df_ff[df_ff.author_id.apply(lambda X: X in baby_set)]
baby_df.to_csv('baby_data.csv', index = False)

baby = baby_df.to_dict(orient='records')

with open('data_vFFFF.pkl', 'wb') as f:
    pkl.dump(baby, f)

In [5]:
len(baby_df)

array([ 2210,  5049,   735,  1252,  5162,  4102,   188,   972,  2384,
         406,  5368,  1208,  1082,  5101,  7850, 21042,    92,  2881,
        2376,  3948,   562,   163, 10609,  5214,  4077, 15815,   190,
        6001,  4356,  5722,  1775,  4626,  1306,  6099,  1591,  1029,
        6262,  4224, 10984,  4618,  2974,  1609, 10482,  5259, 14454,
        2174,  8010,  1644, 25006,  3214,  3671,  2526, 16731,  2482,
        4872,  9803])

In [10]:
pd.DataFrame(X_train, y_train)

NameError: name 'X_train' is not defined

In [ ]:
seed_works[seed_works['id'] == 'PG53940']

,Unnamed: 0,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,subject_set,Sub_A,Sub_B,Sub_C
7975,53938,PG53940,"Haw-Ho-Noo; Or, Records of a Tourist","Lanman, Charles",1819.0,1895.0,['en'],3.0,"{'Fishing -- North America', 'United States --...",Text,"['Description and travel', 'North America', 'F...",Description and travel,North America,Fishing


In [ ]:
for_merge = seed_works[['id', 'authoryearofbirth', 'authoryearofdeath', 'downloads', 'subject_set', 'Sub_A', 'Sub_B', 'Sub_C']]

In [ ]:
ff_df_null = ff_df[ff_df.authoryearofbirth.isna()]
ff_df = ff_df[~ff_df.authoryearofbirth.isna()]

ff_df_null = ff_df_null[['author_id', 'author_name', 'book_id', 'gutenbergbookid', 'title',
       'text', 'text_lines']]
null_values = ff_df_null.merge(for_merge, left_on='book_id', right_on = 'id')
null_values['topic'] = null_values['Sub_A']

In [ ]:
ff_df = pd.concat((ff_df, ff_df_null))

Unique works for each of the three datasets:

In [ ]:

len(local_df.book_id.unique())
# len(final_df.book_id.unique())
len(ff_df.book_id.unique())


6424

Because the final_df has a lot of text from non-English languages, I have decided to proceed as follows:
* Our overall dataset will be the `ff_df` object, corresponding to the `data_vFF.pkl` file. The filtering for this file was the most stringent.
* We will use the works not included in `ff_df`, but in `local_df`, as the training corpus for our word embeddings

In [ ]:
local_df['str_text_lines'] = local_df['text_lines'].apply(str)
local_df['passage_key'] = local_df['gutenbergbookid'] + '_' + local_df['str_text_lines']

ff_df['str_text_lines'] = ff_df['text_lines'].apply(str)
ff_df['passage_key'] = ff_df['gutenbergbookid'] + '_' + ff_df['str_text_lines']


In [ ]:
dataset_passages = list(ff_df.passage_key.unique())

In [ ]:
ldf_index = local_df.set_index('passage_key').index
ff_index = ff_df.set_index('passage_key').index
corpus_index = ldf_index.difference(ff_index)

In [ ]:
local_df_idx = local_df.set_index('passage_key')
corpus_df = local_df_idx.loc[corpus_index].reset_index()

In [ ]:
corpus_df['joined_text'] = corpus_df.text.apply(' '.join)

In [ ]:
corpus_text = " ".join(corpus_df.joined_text)

In [ ]:
with open('corpus_text.txt', 'w') as f:
    f.write(corpus_text)

In [ ]:
os.chdir(config['REPODIR'])

In [ ]:
ff_out = ff_df.to_dict(orient='records')
U.write_file(ff_out, 'data_vFF.pkl', 'pkl', config['DATADIR'])

TypeError: file must have a 'write' attribute